# Notebook 01 — EDA & Cleaning (Days 1–2)

This notebook performs data loading, filtering to 2-3 bedroom apartments, cleaning, exploratory data analysis, missing-value treatment, outlier detection, and saves a stage-1 cleaned dataset and EDA artifacts under `outputs/01_eda/`.  

**Run this notebook start-to-finish.**

In [ ]:
# Dependencies and setup\n\nimport warnings
warnings.filterwarnings('ignore')\n\nimport numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import joblib\n\nRND = 42
np.random.seed(RND)\n\nDATA_PATH = Path(r'/mnt/data/cairo_real_estate_dataset.csv')\n\nOUT_DIR = Path('outputs/01_eda')
OUT_DIR.mkdir(parents=True, exist_ok=True)\n\nprint('Looking for dataset at', DATA_PATH)\n\nif not DATA_PATH.exists():
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Please upload cairo_real_estate_dataset.csv to /mnt/data/.")\n\ndf = pd.read_csv(DATA_PATH, parse_dates=['listing_date'], dayfirst=True)
print('Original shape:', df.shape)
df.head()

## Basic cleaning and type standardization
- Convert Yes/No/TrueFalse booleans to 0/1
- Standardize categorical text fields
- Filter to 2-3 bedroom apartments (target population)

In [ ]:
# Standardize boolean-like columns and basic text fields
bool_cols = ['has_balcony','has_parking','has_security','has_amenities','is_negotiable']
for c in bool_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.lower().map({'yes':1,'no':0,'true':1,'false':0,'1':1,'0':0})

if 'seller_type' in df.columns:
    df['seller_type'] = df['seller_type'].astype(str).str.strip().str.title()
if 'finishing_type' in df.columns:
    df['finishing_type'] = df['finishing_type'].astype(str).str.strip().str.title()

# Filter to 2-3 bedrooms
if 'bedrooms' in df.columns:
    before = df.shape[0]
    df = df[df['bedrooms'].isin([2,3])].copy()
    print(f'Filtered from {before} rows to {df.shape[0]} rows (2-3 bedrooms)')
else:
    print('No bedrooms column found; proceeding without filter')

# Numeric coercion and derived features
for col in ['area_sqm','price_egp']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

if 'area_sqm' in df.columns and 'price_egp' in df.columns:
    df.loc[df['area_sqm'] <= 0, 'area_sqm'] = pd.NA
    df['price_per_sqm'] = df['price_egp'] / df['area_sqm']

if 'listing_date' in df.columns:
    df['listing_month'] = df['listing_date'].dt.month
    df['listing_year'] = df['listing_date'].dt.year

print('After typing and derived features, shape:', df.shape)
df.head()

## Missing value summary and data quality plan

In [ ]:
# Missing values summary
mv = pd.DataFrame({'n_missing': df.isnull().sum(), 'pct_missing': df.isnull().mean()}).sort_values('pct_missing', ascending=False)
mv.to_csv(OUT_DIR / 'missing_values_summary.csv', index=True)
print(mv.head(20))

# Data quality plan (initial)
plan = []
for col in df.columns:
    plan.append({'column': col, 'n_missing': int(df[col].isnull().sum()), 'pct_missing': float(df[col].isnull().mean()), 'proposal': 'investigate and impute/drop/encode'})
plan_df = pd.DataFrame(plan).sort_values('pct_missing', ascending=False)
plan_df.to_csv(OUT_DIR / 'data_quality_plan.csv', index=False)
print('Saved missing value summary and data quality plan to', OUT_DIR)

## Exploratory plots
Saved figures: price distribution, price_per_sqm boxplot, numeric correlation heatmap, top districts and finishing types

In [ ]:
# Ensure numeric columns exist
num_cols = df.select_dtypes(include=['number']).columns.tolist()

# Price histogram
if 'price_egp' in df.columns:
    plt.figure(figsize=(8,5))
    sns.histplot(df['price_egp'].dropna(), bins=60)
    plt.title('Price distribution')
    plt.xlabel('Price (EGP)')
    plt.savefig(OUT_DIR / 'price_hist.png', bbox_inches='tight')
    plt.close()

# Price per sqm boxplot
if 'price_per_sqm' in df.columns:
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df['price_per_sqm'].dropna())
    plt.title('Price per sqm boxplot')
    plt.savefig(OUT_DIR / 'pps_box.png', bbox_inches='tight')
    plt.close()

# Correlation heatmap
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Numeric correlation matrix')
    plt.savefig(OUT_DIR / 'corr_matrix.png', bbox_inches='tight')
    plt.close()

# Categorical counts
if 'district' in df.columns:
    plt.figure(figsize=(10,5))
    df['district'].value_counts().head(20).plot(kind='bar')
    plt.title('Top districts (counts)')
    plt.savefig(OUT_DIR / 'district_counts.png', bbox_inches='tight')
    plt.close()

if 'finishing_type' in df.columns:
    plt.figure(figsize=(8,4))
    df['finishing_type'].value_counts().plot(kind='bar')
    plt.title('Finishing type counts')
    plt.savefig(OUT_DIR / 'finishing_counts.png', bbox_inches='tight')
    plt.close()

print('Saved EDA figures to', OUT_DIR)

## Outlier detection
- Use robust thresholds (1st and 99th percentiles) for price_per_sqm
- Mark outliers but keep them for review

In [ ]:
# Outlier marking
if 'price_per_sqm' in df.columns:
    q1 = df['price_per_sqm'].quantile(0.01)
    q99 = df['price_per_sqm'].quantile(0.99)
    df['pps_outlier'] = ((df['price_per_sqm'] < q1) | (df['price_per_sqm'] > q99)).astype(int)
    print('price_per_sqm 1%:', q1, '99%:', q99)

# Area outliers
if 'area_sqm' in df.columns:
    df['area_outlier'] = ((df['area_sqm'] < 25) | (df['area_sqm'] > 500)).astype(int)

# Save stage1 cleaned dataset
DATA_DIR = Path('data')
DATA_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_PATH = DATA_DIR / 'cleaned_df_stage1.parquet'
df.to_parquet(CLEAN_PATH, index=False)
print('Saved stage1 cleaned dataset to', CLEAN_PATH)

### Next steps
Proceed to Notebook 02 to build preprocessors and baseline models. The stage1 cleaned file is saved to `data/cleaned_df_stage1.parquet`.